# Data PreProcessing 
 Alarm databases in Chemical Plant such as Sonatrach Skikda refinery consist of records of time -Stamped event Logs. since different vendors of modern control systems generate their own format of messages. the format of alarm data is both software and hardware dependent. as a minimum requirement, each record contains fields that store information such as unique indentifier for each alarm Tag,time/date,Alarm Priorities,alarm description which includes Activation Alarm <span style='color:blue'>[ALM]</span> & return <span style='color:blue'>[RTN]</span> .... ect 
 in this machine learning model we gonna train historique data recovred from Sonatrach Skikda refinery / boiler 1051 .using the documentation provided with the P&ID we can identify <br>
 
<b> ALARM STATES:</b>  <br>
  -return Alarm <span style='color:blue'>[NR]</span>  <br>
  -Activation   <span style='color:blue'>[ALM]</span> <br>
  -ALARM VERY HIGH THRESHOLD ON MEASUREMENT <span style='color:blue'>[HH]</span><br>
  -ALARM THRESHOLD VERY LOW ON MEASUREMENT  <span style='color:blue'>[LL]</span><br>
  -ALARM HIGH THRESHOLD ON MEASUREMENT      <span style='color:blue'>[HI]</span><br>
  -ALARM LOW THRESHOLD ON MEASUREMENT       <span style='color:blue'>[LO]</span><br>
  -HIGH LIMIT ALARM ON OUTPUT               <span style='color:blue'>[MHI]</span><br>
  
<b>OPERATING MODES:</b><br>
  -Manual                <span style='color:blue'>[MAN]</span><br>
  -AUTOMATIQUE           <span style='color:blue'>[AUT]</span><br>
  -MANUAL INITIALIZATION <span style='color:blue'>[IMAN]</span><br>
  -FOLLOWING EXIT        <span style='color:blue'>[TRK]</span><br>
  -CASCADE               <span style='color:blue'>[CAS]</span><br>
 
<b>OTHER:</b> <br>
  -RECOVER ALARM SIGNAL AFTER A LOSS  <span style='color:blue'>[RECOVER]</span><br>
  -ALARM THAT OCCURE ONLY TO THIS USER[<span style='color:blue'>ONUSER..]</span><br>

 We assume that a possible Alarm Sequence could be <span style='color:blue'>[ALM]->[NR]</span> and not <span style='color:blue'>[ALM]->[ALM]->[NR]</span> during this phase its extremely important to carefully investigate and prepare this data since the real plant data tends to be inconsistent with errors, missing values,duplicate values. <br>
 overall mining results depend on the data quality thus it is essential to train this model to find the correlation between this alarms <br>
 this step inculdes removal of irrelavant records such as all the rows in the database containing {OPERATING MODES} such as <span style='color:blue'>[AUT]</span> ....ect <br>
 Removal of redundant data such as multiple <span style='color:blue'>[ALMs]</span> wih identical time stamps or data with missing <span style='color:blue'>[NR]</span> and remove the recover of alarm signal <span style='color:blue'>[RECOVER]</span> <br>

 *Finally* an alarm squence is create where each event has an associated time of occurrence given as an integer number , and corresponding event type represented as postive or negative integer. <br>
 
 
 
 
 
       
        

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib import pyplot
import xml.etree.ElementTree as ET



In [2]:
data = pd.read_csv('one.csv')
data.info()
#Book2
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7773 entries, 0 to 7772
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    7773 non-null   object
 1   Heure   7773 non-null   object
 2   Tag     7773 non-null   object
 3   Alarm   7773 non-null   object
dtypes: object(4)
memory usage: 243.0+ KB


,Date,Heure,Tag,Alarm
0,2022/02/01 00:00:06.340 +01:00,3:02:00,1051PI5318,1051PI5318 AIR BLOWER TO F.S. PRESS PV = 7...
1,2022/02/01 00:00:07.340 +01:00,3:02:00,1051PI5318,1051PI5318 AIR BLOWER TO F.S. PRESS NR
2,2022/02/01 00:00:37.040 +01:00,5:02:00,1051IL4453AL,1051IL4453AL INTERLOCK ALARM NR
3,2022/02/01 00:00:40.040 +01:00,5:02:00,1051PALL5318,1051PALL5318 AIR BLOWER TO F.S. PRESS NR
4,2022/02/01 00:01:27.290 +01:00,3:00:00,1051FV5313A,1051FV5313A 1ST COMB. AIRFAN F.C.D MV = ...
5,2022/02/01 00:01:29.290 +01:00,3:02:00,1051FI5312A,1051FI5312A COMBUSTION AIR FLOW Ch A PV = 3...
6,2022/02/01 00:03:55.290 +01:00,3:02:00,1051FI5312B,1051FI5312B COMBUSTION AIR FLOW Ch B PV = 3...
7,2022/02/01 00:03:58.290 +01:00,3:02:00,1051FI5312C,1051FI5312C COMBUSTION AIR FLOW Ch C PV = 3...
8,2022/02/01 00:04:16.340 +01:00,3:02:00,1051FI5312,1051FI5312 COMBUSTION AIR AVE-FLOW PV = 3...
9,2022/02/01 00:04:18.290 +01:00,3:02:00,1051FI5312A,1051FI5312A COMBUSTION AIR FLOW Ch A PV = 4...


In [3]:
class Preprocecssing:
    
    def NumTag(Dframe):
        from sklearn.preprocessing import LabelEncoder
        le = LabelEncoder()
        X = le.fit_transform(Dframe['Tag'])
        Dframe['NumTag']=X
        return Dframe
    
    def checkNumTag(Dframe):
        DfALL=Dframe.index[Dframe["Tag"].astype("str").str.contains("1")].tolist()  
        for i in DfALL:
            if Dframe.at[i,'NumTag'].astype(int) == 0:
                Dframe.at[i,'NumTag']=Dframe.at[i,'NumTag'].astype(int)+99            
        return Dframe
    def tagFault(Dframe):
        x=Dframe['Alarm']
        Dframe['Fault']=x
        return Dframe
    def AlarmState(Dframe):
        x=Dframe.index[Dframe["Alarm"].astype("str").str.contains(" NR")].tolist()
        r0=Dframe.index[Dframe["Alarm"].astype("str").str.contains("ONUSER")].tolist()
        r1=Dframe.index[Dframe["Alarm"].astype("str").str.contains(" AUT")].tolist()
        y7=Dframe.index[Dframe["Alarm"].astype("str").str.contains(" DegC ")].tolist()
        r3=Dframe.index[Dframe["Alarm"].astype("str").str.contains(" Recover")].tolist()
        r4=Dframe.index[Dframe["Alarm"].astype("str").str.contains(" IMAN")].tolist()
        r5=Dframe.index[Dframe["Alarm"].astype("str").str.contains(" CAS")].tolist()
        r6=Dframe.index[Dframe["Alarm"].astype("str").str.contains(" TRK")].tolist()
        r7=Dframe.index[Dframe["Alarm"].astype("str").str.contains(" MAN")].tolist()  
        y0=Dframe.index[Dframe["Alarm"].astype("str").str.contains(" HH")].tolist()
        y1=Dframe.index[Dframe["Alarm"].astype("str").str.contains(" HI")].tolist()
        y2=Dframe.index[Dframe["Alarm"].astype("str").str.contains(" LL")].tolist()
        y3=Dframe.index[Dframe["Alarm"].astype("str").str.contains(" LO")].tolist()
        y4=Dframe.index[Dframe["Alarm"].astype("str").str.contains(" ALM")].tolist()
        y5=Dframe.index[Dframe["Alarm"].astype("str").str.contains(" NR")].tolist()
        y6=Dframe.index[Dframe["Alarm"].astype("str").str.contains(" MHI")].tolist()

        r=r0+r1+r3+r4+r5+r6+r7
        y=y0+y1+y2+y3+y4+y5+y6+y7
        
        for i in x:
            Dframe.at[i,'Alarm']=  Dframe.at[i,'NumTag'].astype(int)*-1
        for j in y:
            Dframe.at[j,'Alarm']=  Dframe.at[j,'NumTag'].astype(int)*1
        for i in x:
            Dframe.at[i,'Alarm']=  Dframe.at[i,'NumTag'].astype(int)*-1
     
        for z in r:
            Dframe.at[z,'Alarm']= np.nan
            
        del Dframe["NumTag"]
        return Dframe;
            
         
    def SequenceAlarm(DfAlarm):
       
        DfAlarm =DfAlarm.reset_index(drop=True)
        DfALL=DfAlarm.index[DfAlarm["Tag"].astype("str").str.contains("1")].tolist()  
        for i in DfALL:        
            if(i==0):
                x=1                 
            else:
                if DfAlarm.at[i,'Alarm'].astype(int) == DfAlarm.at[i-1,'Alarm'].astype(int) :
                    DfAlarm=DfAlarm.drop(i-1)
        DfAlarm =DfAlarm.reset_index(drop=True)
        return DfAlarm
    
    def AlarmTime(Dframe):
        Dframe.insert(2, 'Time', range(1, 1 + len(Dframe)))
        return Dframe
    
        
    

In [4]:
DFAlarms = data.drop(['Heure'],axis=1)
DfAlarm=Preprocecssing.NumTag(DFAlarms)
DfAlarm=Preprocecssing.checkNumTag(DfAlarm)
DfAlarm=Preprocecssing.tagFault(DfAlarm)
DfAlarm=Preprocecssing.AlarmState(DFAlarms)
DfAlarm.dropna(inplace=True)
DfAlarm.to_csv('out.zip', index=False)  
DfAlarm =DfAlarm.reset_index(drop=True)
DfAlarm=Preprocecssing.SequenceAlarm(DfAlarm)
DfAlarm =DfAlarm.reset_index(drop=True)
DfAlarm=Preprocecssing.AlarmTime(DfAlarm)
#y=y.drop(['Tag'],axis=1)
DfAlarm.head(100).style.set_properties(subset=['Alarm'], **{'width-min': '700px'})



,Date,Tag,Time,Alarm,Fault
0,2022/02/01 00:00:07.340 +01:00,1051PI5318,1,-59,1051PI5318 AIR BLOWER TO F.S. PRESS NR
1,2022/02/01 00:00:37.040 +01:00,1051IL4453AL,2,-42,1051IL4453AL INTERLOCK ALARM NR
2,2022/02/01 00:00:40.040 +01:00,1051PALL5318,3,-56,1051PALL5318 AIR BLOWER TO F.S. PRESS NR
3,2022/02/01 00:04:59.290 +01:00,1051FI5312A,4,-6,1051FI5312A COMBUSTION AIR FLOW Ch A NR
4,2022/02/01 00:06:03.290 +01:00,1051FI5312B,5,-7,1051FI5312B COMBUSTION AIR FLOW Ch B NR
5,2022/02/01 00:06:26.340 +01:00,1051FI5312C,6,-8,1051FI5312C COMBUSTION AIR FLOW Ch C NR
6,2022/02/01 00:06:44.040 +01:00,1051FI5312,7,-5,1051FI5312 COMBUSTION AIR AVE-FLOW NR
7,2022/02/01 00:06:45.040 +01:00,1051FSLL5312_AL,8,14,1051FSLL5312 FSL-5312-COMBUSTION AIR ALM
8,2022/02/01 00:08:07.340 +01:00,1051FI5313,9,-9,1051FI5313 COMBUSTION AIR FLOW NR
9,2022/02/01 00:11:13.040 +01:00,1051PI5311,10,-58,1051PI5311 COMBUSTION AIR PRESSURE NR


In [8]:
dups = DFAlarms.pivot_table(index = ['Tag'], aggfunc ='size')
dups

Tag
1051AI5311          3
1051AI5391          6
1051AI5392          6
1051AIC5311         2
1051FALL5312_AL     4
                   ..
1051ZAL5332         2
1051ZAL5333        58
1051ZAL5335        14
1051ZAL5336        12
1051ZAL5337        14
Length: 96, dtype: int64

In [11]:
newdf = DFAlarms.drop_duplicates(
  subset = ['Tag'],
  keep = 'last').reset_index(drop = True)
newdf

,Date,Tag,Alarm,Fault
0,2022/02/01 01:24:43.040 +01:00,1051XA5311,-88,1051XA5311 BMS FAILURE NR
1,2022/02/01 01:27:52.040 +01:00,1051AIC5311,-3,1051AIC5311 O2 ANALYZER IN F.G.S. NR
2,2022/02/01 01:28:29.040 +01:00,1051AI5311,-99,1051AI5311 O2 ANALYZER IN F.G.S. NR
3,2022/02/01 01:28:47.090 +01:00,1051XA5308,-85,1051XA5308 BURNER#3 ON FOR G3 NR
4,2022/02/01 01:28:58.340 +01:00,1051ZAH5332,105,1051ZAH5332 B3-GAS V/V 2 ZSH5332 NOP ALM
...,...,...,...,...
91,2022/02/01 10:30:11.040 +01:00,1051LI5303B,46,1051LI5303B STEAM DRUM LEVEL Ch B PV = ...
92,2022/02/01 10:30:32.040 +01:00,1051TI5309B,-75,1051TI5309B SH2 TEMP. B NR
93,2022/02/01 10:31:01.040 +01:00,1051LI5304,48,1051LI5304 STEAM DRUM LEVEL PV = ...
94,2022/02/01 10:31:01.000 +01:00,1051LIC5304B,50,1051LIC5304B STEAM DRUM LEVEL PV = ...


# Teloiral Windows
  <script async="true" src="https://cdn.jsdelivr.net/npm/mathjax@2/MathJax.js?config=AM_CHTML"> </script>
<div>
<img src="./Capture.JPG" width="500"/>
</div>


<p>the example in the previous picture shows is this method thers two sets of temporal relations need to be taken into account.since its difficult to decide how close is close enough when giving the width of the (R-w) window as it depends on the size as well as the cause significance of the time intervals</p><p> if the return of (R-A) is too short we cannot observe relationships as other associated events will be still in an active state thus rather than construscting a sliding window for each actionvation and return event we join together consecutive windows until the stopping criterion is met.</p><p> Suppose we are givena sequence S, an integer n representing the total number of activations of a specified alarm tag and a retun window width w we form s all windows  `W_(R-A/R-w(1)),W_(R-A/R-w(2)),....W_(R-A/R-w(i)) Assume we duration i=1,2,3....i have a time intervall and user give minimum time interver MIN. <br/>the time interval MIn can be used for more efficient segmentation of
 W_A-R and W_R-A/R-W  subsequence into corresponding activation and retun transactional windows respectively, in the case of win < minw .  </p>
 <div>
<img src="./1.png" width="500"/>
</div>

In [22]:
class Algorithm:
    def GetAlarms(DfAlarm):
        Rec= DfAlarm['Alarm']
        Rec=list(dict.fromkeys(DfAlarm['Alarm']))
        Rec=list(filter(lambda x: x > 0, Rec))
        return Rec
    def GetAlarmss(DfAlarm):
        Rec=list(dict.fromkeys(DfAlarm['Alarm']))
        Rec=list(filter(lambda x: x < 0, Rec))
        return Rec
    
    def calcul(DfAlarm,rec,x):
        DfALL=DfAlarm.index[DfAlarm["Tag"].astype("str").str.contains("1")].tolist()
        m=[]
        l=[]
        Flag=False
        for i in DfALL:
            if DfAlarm.at[i,'Alarm'].astype(int) == x:
                Flag=True
            else:
                if DfAlarm.at[i,'Alarm'].astype('int')== -x:
                    Flag=False
            if DfAlarm.at[i,'Alarm'].astype('int') != x and DfAlarm.at[i,'Alarm'].astype('int') != -x and Flag==True:
                l.append((DfAlarm.at[i,'Alarm'].astype('int'))) 
         
            else:
                    if l :
                        m.append(l)
                        l=[]
                        Flag=False
                
                
                    
        return m
    
    def user_calcul(DfAlarm,rec,x,t):
        DfALL=DfAlarm.index[DfAlarm["Tag"].astype("str").str.contains("1")].tolist()
        m=[]
        l=[]
        t1=0
        Flag=False
        for i in DfALL:
            if DfAlarm.at[i,'Alarm'].astype(int) == x:
                Flag=True
                t1=1
            else:
                if DfAlarm.at[i,'Alarm'].astype('int')== -x or t1>=t:
                    Flag=False
                    t1=1
            if DfAlarm.at[i,'Alarm'].astype('int') != x and DfAlarm.at[i,'Alarm'].astype('int') != -x and Flag==True:
                l.append((DfAlarm.at[i,'Alarm'].astype('int'))) 
                t1=t1+1
         
            else:
                    if l :
                        t1=1
                        m.append(l)
                        l=[]
                        Flag=False
                
                
                    
        return m
        

# Rule Discovery from Transactions
<p>
we focus in the step in finding local realtionships from the list of transactions obtained in the data segmentation and filtering phase. we favorate the idea of the closer toghther the cause and consequence alarms in time.
</p>
<p>
Exmple: if alarm A occured, then (B and C)  will occur within time T_E we write A=>(B,C,T_E).
<div>
<img src="./Capture1.jpg" width="300"/>
</div>
where ferq(A,B,C,T_E)={i|a_i=(A Ʌ B Ʌ C)ЁT_E(i)} is the number of occurences of B and C that occure after A with in time interval 
</p>
<p>
calculating the confidences of target episodes is not diffcult since tha alarm sequence is segmented into sets of transactions associated with each specific alarm tag.
</p>

In [16]:
class Rule_Discovry:
    def DictSequences(DfAlarm):
        r=[]
        for i in DfAlarm:
            res = list(filter(lambda x : x > 0, i))
            r.append(res)
        return r
    
    def fill_tag(df,DfAlarm):
        m=[]
        rec=Algorithm.GetAlarms(DfAlarm)
        flag=True
        for i in rec:  
            for j in DfAlarm['Time']:             
                if (i==DfAlarm.at[j-1,'Alarm'].astype(int))  and flag==True:
                    m.append(DfAlarm.at[j-1,'Fault'])
                    flag=False                 
            flag=True    
        df['ALARM']=m
        df['Tag']=rec
        return df
    
    def calcule_rols(x,df,DfAlarm):
        r=[]
        rec=Algorithm.GetAlarms(DfAlarm)
        m=Algorithm.calcul(DfAlarm,rec,x)
        m=Rule_Discovry.DictSequences(m)
        print(m)
        freq=DfAlarm['Alarm'].values.tolist()
        for i in m :
            conf= (m.count(i)/freq.count(x))*100
            conf=round(conf, 2)
            strr='',x,' => ',i,'  ',conf,')'
            r.append(strr);
        return r
    
    def user_calcule_rols(x,df,DfAlarm):
        r=[]
        rec=Algorithm.GetAlarms(DfAlarm)
        m=Algorithm.user_calcul(DfAlarm,rec,x,4)
        m=Rule_Discovry.DictSequences(m)
        print(m)
        freq=DfAlarm['Alarm'].values.tolist()
        for i in m :
            conf= (m.count(i)/freq.count(x))*100
            conf=round(conf, 2)
            strr='',x,' => ',i,'  ',conf,')'
            r.append(strr);
        return r
    
    def to_dataframe(x,r,df,pos):
        x1=df.index[df['Tag'] == x].tolist()
        retruned=[]
        for i in r:
            m=''
            p=0
            for j in i:
                if p<6:
                    m=m+str(j) 
                if p==6:
                    m=m+"      "
                    retruned.append(m)
                    p=0;
                p+=1    
               
        my_final_list = set(retruned)
        for i in my_final_list:
            if float(i[-11:-3]) < 100:
                m=0;        
                if(m<float(i[-11:-3])):
                    m=float(i[-11:-3])
                    for s in my_final_list:
                        if(float(s[-11:-3])==m):
                            first_slid=s
                for r in x1:
                    if(pos=='slid'):
                        df.at[r,'Simple_Sliding %']= first_slid
                        s=''
                    else:
                        df.at[r,'User_Sliding %']= first_slid
                        s=''
                    df.at[r,'Proposed']= 1
        
        return df
    
    def proposed(x,df):
        x1=df.index[df['Tag'] == x].tolist()
        for r in x1:
            if((float(df.at[r,'Simple_Sliding %'][-11:-3]))<=(float(df.at[r,'User_Sliding %'][-11:-3]))):
                df.at[r,'Proposed']=df.at[r,'User_Sliding %']
            else:
                df.at[r,'Proposed']=df.at[r,'Simple_Sliding %']
        return df
                
        
    
    
    

In [7]:

df = pd.DataFrame({'ALARM':[],'Tag': [],'Simple_Sliding %': [],'User_Sliding %': [],'Proposed': []})
df=Rule_Discovry.fill_tag(df,DfAlarm)


In [23]:
r1=Rule_Discovry.user_calcule_rols(47,df,DfAlarm)

[[44, 45, 58], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [], [44], [44], [44], [44], [44, 46, 44], [44], [44, 45, 46], [44], [44], [44, 45, 46], [44], [44], [44, 45, 46], [44, 45, 46], [44, 45, 46], [44], [44, 45, 46], [44, 45, 46], [44, 45, 46], [44, 45, 46], [44, 45, 46], [44, 76, 45], [44], [44, 45, 46], [44, 45, 46], [], [44], [44], [44, 46, 44], [], [44, 45, 46], [44], [44], [44], [44, 46], [44], [44], [44], [], [44, 45, 46], [44], [44], [], [44], [44], [44], [44], [44], [44], [44], [44], [44], [], [44], [44], [44, 48], [44], [44], [44], [44], [], [44, 46], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44, 50, 49], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [], [44], [], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44], [44

In [9]:

for i in df['Tag']:
    r=Rule_Discovry.calcule_rols(int(i),df,DfAlarm)
    r1=Rule_Discovry.user_calcule_rols(int(i),df,DfAlarm)
    df=Rule_Discovry.to_dataframe(int(i),r,df,'slid')
    df=Rule_Discovry.to_dataframe(int(i),r1,df,'user')
df.dropna(inplace=True)  
for i in df['Tag']:
    df=Rule_Discovry.proposed(int(i),df)

    
    

In [10]:
df.sort_values(by=['Tag'])

,ALARM,Tag,Simple_Sliding %,User_Sliding %,Proposed
15,1051AI5391 BFW TOTAL CONDUCTIVITY PV = ...,1,"1 => [87, 62, 63, 61, 67, 65, 11, 64, 10, 13, ...","1 => [110, 97] 33.33","1 => [110, 97] 33.33"
60,1051AI5392 BFW PH PV = ...,2,"2 => [50, 49, 22, 42, 56, 59, 23, 14, 117, 119...","2 => [20, 6, 7, 8] 33.33","2 => [20, 6, 7, 8] 33.33"
70,1051FALL5312 COMBUSTION AIR FLL LOW ALM,4,"4 => [81, 9, 82, 6, 7, 8, 5, 95, 14, 58, 7, 8,...","4 => [81, 9] 50.0","4 => [81, 9] 50.0"
68,1051FI5312 COMBUSTION AIR AVE-FLOW PV = 3...,5,"5 => [9, 4, 81, 9, 82, 6, 7, 8] 16.67","5 => [6, 9, 8, 7] 16.67","5 => [6, 9, 8, 7] 16.67"
65,1051FI5312A COMBUSTION AIR FLOW Ch A PV = 3...,6,"6 => [7, 8, 5, 21] 16.67",6 => [94] 16.67,6 => [94] 16.67
...,...,...,...,...,...
77,1051ZAL5331 B3-GAS V/V 1 ZSL-5331 ALM,117,"117 => [119, 106] 66.67","117 => [119, 106, 87] 11.11","117 => [119, 106] 66.67"
78,1051ZAL5333 B3-GAS VENT V ZSL5333 NC ALM,119,"119 => [106, 90, 67, 11, 10, 13, 110, 97, 84, ...","119 => [106, 62, 64, 61] 3.45","119 => [106, 62, 64, 61] 3.45"
39,1051ZAL5335 B4-GAS V/V1 ZSL-5335 ALM,120,"120 => [108, 122, 109, 121, 58, 90, 46] 14.29...","120 => [122, 109, 45, 46] 14.29","120 => [122, 109, 45, 46] 14.29"
43,1051ZAL5336 B4-GAS V/V 2 ZSL-5336 ALM,121,"121 => [90, 90, 109, 90, 88, 88, 90, 75, 90, 9...",121 => [90] 16.67,121 => [90] 16.67


In [11]:
df.to_csv('zip.zip', index=False)  

In [12]:
cols = ['ALARM','Tag']
df1 = pd.DataFrame(df, columns=cols)


In [15]:
df1.sort_values(by=['Tag'])

,ALARM,Tag
15,1051AI5391 BFW TOTAL CONDUCTIVITY PV = ...,1
60,1051AI5392 BFW PH PV = ...,2
70,1051FALL5312 COMBUSTION AIR FLL LOW ALM,4
68,1051FI5312 COMBUSTION AIR AVE-FLOW PV = 3...,5
65,1051FI5312A COMBUSTION AIR FLOW Ch A PV = 3...,6
...,...,...
77,1051ZAL5331 B3-GAS V/V 1 ZSL-5331 ALM,117
78,1051ZAL5333 B3-GAS VENT V ZSL5333 NC ALM,119
39,1051ZAL5335 B4-GAS V/V1 ZSL-5335 ALM,120
43,1051ZAL5336 B4-GAS V/V 2 ZSL-5336 ALM,121


In [16]:
df1.to_csv('zip1.zip', index=False)  